In [2]:
!conda install -c conda-forge folium=0.5.0 --yes
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                       

In [17]:
CLIENT_ID = 'QWC0YZGWKDKSNIAYOFV4S0O1QZ53XJ5OGBESH2KCXIT5Q3IT' # your Foursquare ID
CLIENT_SECRET = 'P4GZI2VDT1VUBUWC2MOMGTMIJRZNQBG4I0WJYUUMJ3HFNNZO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: QWC0YZGWKDKSNIAYOFV4S0O1QZ53XJ5OGBESH2KCXIT5Q3IT


In [18]:
# type your answer here
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Jersey City, NJ', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735") # PIZZA PLACE CATEGORY ID
    results[city] = requests.get(url).json()
results

{'New York, NY': {'meta': {'code': 200,
   'requestId': '5e9f04f569babe001b7c84bd'},
  'response': {'suggestedFilters': {'header': 'Tap to show:',
    'filters': [{'name': 'Open now', 'key': 'openNow'},
     {'name': '$-$$$$', 'key': 'price'}]},
   'geocode': {'what': '',
    'where': 'new york ny',
    'center': {'lat': 40.742185, 'lng': -73.992602},
    'displayString': 'New York, NY, United States',
    'cc': 'US',
    'geometry': {'bounds': {'ne': {'lat': 40.882214, 'lng': -73.907},
      'sw': {'lat': 40.679548, 'lng': -74.047285}}},
    'slug': 'new-york-city-new-york',
    'longId': '72057594043056517'},
   'headerLocation': 'New York',
   'headerFullLocation': 'New York',
   'headerLocationGranularity': 'city',
   'query': 'pizza',
   'totalResults': 260,
   'suggestedBounds': {'ne': {'lat': 40.79342318962265,
     'lng': -73.91554876647989},
    'sw': {'lat': 40.68819722406004, 'lng': -74.01711040667074}},
   'groups': [{'type': 'Recommended Places',
     'name': 'recommended'

In [19]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [20]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of pizza places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of pizza places in New York, NY =  260
Showing Top 100
Total number of pizza places in Chicago, IL =  214
Showing Top 100
Total number of pizza places in San Francisco, CA =  170
Showing Top 100
Total number of pizza places in Jersey City, NJ =  120
Showing Top 100
Total number of pizza places in Boston, MA =  187
Showing Top 100


In [21]:
maps[cities[0]]

In [22]:
maps[cities[1]]

In [23]:
maps[cities[2]]

In [24]:
maps[cities[3]]

In [25]:
maps[cities[4]]

In [26]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.021613494982558414
Chicago, IL
Mean Distance from Mean coordinates
0.05951214771607171
San Francisco, CA
Mean Distance from Mean coordinates
0.029482815390110426
Jersey City, NJ
Mean Distance from Mean coordinates
0.01923432811952773
Boston, MA
Mean Distance from Mean coordinates
0.03585525093215225


In [27]:
maps[cities[0]]

In [28]:
maps[cities[1]]

In [29]:
maps[cities[2]]

In [30]:
maps[cities[3]]

In [31]:
maps[cities[4]]

We now see that New York is his best option. And as aplus the Third best place is Jersey City which is just on the other side of the shore. Our tourist's best interest would be to book a hotel near that mean coordinate to surround himself with the 100 Pizza stores there!!
Another observation is that there is one really far away Pizza store which would possible increase its score to be beaten by New York So let's try to remove it and calculate it again